# Skorch-Tutorial
- To learn skorch
- To test pre-commit

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Classfication

In [ ]:
from skorch import NeuralNetClassifier

In [ ]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

In [ ]:
class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X))
        return X

In [ ]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.1,
    iterator_train__shuffle=True,  # Shuffle training data on each epoch
    device=device,  # to use cuda:0
)

In [ ]:
net.fit(X, y)
y_proba = net.predict_proba(X)

In [ ]:
y_pred = net.predict(X)

# Regression, Callbacks
- Passing a string: This should be a valid sklearn metric. For a list of all existing scores, look here.
- Passing None: If you implement your own .score method on your neural net, passing scoring=None will tell skorch to use that.
- Passing a function or callable: If we want to define our own scoring function, we pass a function with the signature func(model, X, y) -> score, which is then used.

In [3]:
from skorch import NeuralNetRegressor
from skorch.callbacks import EpochScoring

In [4]:
X_regr, y_regr = make_regression(1000, 20, n_informative=10, random_state=0)
X_regr = X_regr.astype(np.float32)
y_regr = y_regr.astype(np.float32) / 100
y_regr = y_regr.reshape(-1, 1)

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


def root_mean_squared_error(net, ds, y=None):
    # assume ds yields (X, y), e.g. torchvision.datasets.MNIST
    y_true = [y for _, y in ds]
    y_pred = net.predict(ds)

    return np.sqrt(np.mean((y_pred - y_true) ** 2))


mse = EpochScoring(mean_squared_error, lower_is_better=True)
mae = EpochScoring(mean_absolute_error, lower_is_better=True)
rmse = EpochScoring(root_mean_squared_error, lower_is_better=True)

In [11]:
class RegressorModule(nn.Module):
    def __init__(
        self, num_units=10, nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

In [12]:
net_regr = NeuralNetRegressor(
    RegressorModule, max_epochs=20, lr=0.1, device=device, callbacks=[mse, mae, rmse]
)

In [13]:
net_regr.fit(X_regr, y_regr)

  epoch    mean_absolute_error    mean_squared_error    root_mean_squared_error    train_loss    valid_loss     dur
-------  ---------------------  --------------------  -------------------------  ------------  ------------  ------
      1                 1.4750                3.6311                     1.9055        4.5859        3.6311  0.0211
      2                 0.8270                1.3089                     1.1441        3.3932        1.3089  0.0212
      3                 0.5631                0.4663                     0.6829        0.9144        0.4663  0.0209
      4                 0.3268                0.1823                     0.4270        0.3117        0.1823  0.0206
      5                 0.5618                0.4317                     0.6571        0.2939        0.4317  0.0208
      6                 0.3464                0.1892                     0.4349        0.3221        0.1892  0.0209
      7                 0.3385                0.1704                    

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=RegressorModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=1, bias=True)
  ),
)

In [14]:
net_regr.predict(X_regr)

NameError: name 'net_regit' is not defined